Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [6]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.727326
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.752411
Minibatch accuracy: 50.0%
Validation accuracy: 54.9%
Minibatch loss at step 100: 1.254394
Minibatch accuracy: 62.5%
Validation accuracy: 74.6%
Minibatch loss at step 150: 0.942438
Minibatch accuracy: 62.5%
Validation accuracy: 76.2%
Minibatch loss at step 200: 0.217297
Minibatch accuracy: 100.0%
Validation accuracy: 78.0%
Minibatch loss at step 250: 1.498838
Minibatch accuracy: 50.0%
Validation accuracy: 79.2%
Minibatch loss at step 300: 1.269087
Minibatch accuracy: 62.5%
Validation accuracy: 80.0%
Minibatch loss at step 350: 0.519560
Minibatch accuracy: 81.2%
Validation accuracy: 80.0%
Minibatch loss at step 400: 0.653931
Minibatch accuracy: 87.5%
Validation accuracy: 77.5%
Minibatch loss at step 450: 0.617276
Minibatch accuracy: 81.2%
Validation accuracy: 80.9%
Minibatch loss at step 500: 0.511063
Minibatch accuracy: 87.5%
Validation accuracy: 81.2%
M

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [173]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1)) #[5,5,1,16]
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1)) #[5,5,16,16]
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1)) 
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden])) 
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1)) 
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    print (conv)
    pool = tf.nn.max_pool(conv, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    print (pool)
    hidden = tf.nn.relu(pool + layer1_biases)
    print (hidden)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
    pool = tf.nn.max_pool(conv, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(pool + layer2_biases)
    print(hidden)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

Tensor("Conv2D:0", shape=(16, 28, 28, 16), dtype=float32)
Tensor("MaxPool:0", shape=(16, 14, 14, 16), dtype=float32)
Tensor("Relu:0", shape=(16, 14, 14, 16), dtype=float32)
Tensor("Relu_1:0", shape=(16, 7, 7, 16), dtype=float32)
Tensor("Conv2D_2:0", shape=(10000, 28, 28, 16), dtype=float32)
Tensor("MaxPool_2:0", shape=(10000, 14, 14, 16), dtype=float32)
Tensor("Relu_3:0", shape=(10000, 14, 14, 16), dtype=float32)
Tensor("Relu_4:0", shape=(10000, 7, 7, 16), dtype=float32)
Tensor("Conv2D_4:0", shape=(10000, 28, 28, 16), dtype=float32)
Tensor("MaxPool_4:0", shape=(10000, 14, 14, 16), dtype=float32)
Tensor("Relu_6:0", shape=(10000, 14, 14, 16), dtype=float32)
Tensor("Relu_7:0", shape=(10000, 7, 7, 16), dtype=float32)


In [174]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.833130
Minibatch accuracy: 6.2%
Validation accuracy: 11.3%
Minibatch loss at step 50: 1.938861
Minibatch accuracy: 37.5%
Validation accuracy: 43.1%
Minibatch loss at step 100: 1.511103
Minibatch accuracy: 31.2%
Validation accuracy: 69.0%
Minibatch loss at step 150: 1.020329
Minibatch accuracy: 56.2%


KeyboardInterrupt: 

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [188]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64
numConcat = 3

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth*numConcat]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth*numConcat, depth*numConcat], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth*numConcat]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth*numConcat, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  weights5 = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  weights3 = tf.Variable(tf.truncated_normal(
      [3, 3, depth, depth], stddev=0.1))
  weights1 = tf.Variable(tf.truncated_normal(
      [1, 1, num_channels, depth]))
  weightsPool = tf.Variable(tf.truncated_normal(
      [1, 1, num_channels, depth]))

  def inceptionModule(input):
    pool = tf.nn.avg_pool(input, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    poolOne = tf.nn.conv2d(pool, weightsPool, [1, 1, 1, 1], padding='SAME')
    oneXone = tf.nn.conv2d(input, weights1, [1, 1, 1, 1], padding='SAME')
    fiveXfive = tf.nn.conv2d(oneXone, weights5, [1, 2, 2, 1], padding='SAME')
    threeXthree = tf.nn.conv2d(oneXone, weights3, [1, 2, 2, 1], padding='SAME')
    return tf.concat([poolOne, threeXthree, fiveXfive], 3)
  
  def poolConv(data):
    #pool = tf.nn.avg_pool(data, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    conv = tf.nn.conv2d(data, weights1, [1, 1, 1, 1], padding='SAME') #5x5 convolution
    pool = tf.nn.max_pool(conv, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    return pool
    
  # Model.
  def model(data):                         
    #conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    #pool = tf.nn.max_pool(conv, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    #pool = poolConv(data)
    inception = inceptionModule(data)
    hidden = tf.nn.relu(inception + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
    pool = tf.nn.max_pool(conv, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    #pool = poolConv(hidden)
    hidden = tf.nn.relu(pool + layer2_biases)
    print (hidden)
    shape = hidden.get_shape().as_list()
    print(shape)
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases, hidden
  
  #inceptionModule(tf_train_dataset)
  # Training computation.
  logits, hidden = model(tf_train_dataset)
  #drop_hidden = tf.nn.dropout(hidden, keep_prob=0.5)
  #drop_logits = tf.matmul(drop_hidden, layer4_weights) + layer4_biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  #global_step = tf.Variable(0)
  #learning_rate = tf.train.exponential_decay(0.05, global_step, 1, 0.9999)
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_logits, valid_hidden = model(tf_valid_dataset)
  valid_prediction = tf.nn.softmax(valid_logits)
  test_logits, test_hidden = model(tf_test_dataset)
  test_prediction = tf.nn.softmax(test_logits)

Tensor("Relu_1:0", shape=(16, 7, 7, 48), dtype=float32)
[16, 7, 7, 48]
Tensor("Relu_4:0", shape=(10000, 7, 7, 48), dtype=float32)
[10000, 7, 7, 48]
Tensor("Relu_7:0", shape=(10000, 7, 7, 48), dtype=float32)
[10000, 7, 7, 48]


In [189]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 8.769764
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.477741
Minibatch accuracy: 50.0%
Validation accuracy: 39.3%
Minibatch loss at step 100: 1.594503
Minibatch accuracy: 43.8%
Validation accuracy: 66.8%
Minibatch loss at step 150: 0.687911
Minibatch accuracy: 68.8%
Validation accuracy: 74.4%
Minibatch loss at step 200: 0.424118
Minibatch accuracy: 81.2%
Validation accuracy: 77.6%
Minibatch loss at step 250: 1.364311
Minibatch accuracy: 56.2%
Validation accuracy: 77.1%
Minibatch loss at step 300: 0.758614
Minibatch accuracy: 75.0%
Validation accuracy: 78.5%
Minibatch loss at step 350: 0.376142
Minibatch accuracy: 87.5%
Validation accuracy: 80.7%
Minibatch loss at step 400: 0.790134
Minibatch accuracy: 75.0%
Validation accuracy: 79.4%
Minibatch loss at step 450: 0.462329
Minibatch accuracy: 81.2%
Validation accuracy: 80.8%
Minibatch loss at step 500: 0.523807
Minibatch accuracy: 75.0%
Validation accuracy: 80.7%
M